In [1]:
# Dependencies
import pandas as pd
import json
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3

In [2]:
from sqlite3 import Error

In [3]:
# create function to try and connect to a sqlite database.  if it doesn't work, create one...
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

        

In [4]:
if __name__ == '__main__':
    create_connection("..\db\happiness.sqlite")

2.6.0


In [5]:
engine = create_engine("sqlite:///../db/happiness.sqlite")
conn = engine.connect()

## The section below processes the happiness CSV files into SQLite

In [6]:
# read in happiness files for all years
happiness_file_15 = "happiness_2015.csv"
happy15_df = pd.read_csv(happiness_file_15)

happiness_file_16 = "happiness_2016.csv"
happy16_df = pd.read_csv(happiness_file_16)

happiness_file_17 = "happiness_2017.csv"
happy17_df = pd.read_csv(happiness_file_17)

# read in region and continents data
region_cont = "region_continent.csv"
region_cont_df = pd.read_csv(region_cont)


In [7]:
# Create YEAR and UNIQUE_ID fields
# 2015
happy15_df['YEAR'] = 2015
happy15_df['UNIQUE_ID'] = happy15_df['COUNTRY'] + '_' + happy15_df.YEAR.map(str)

# 2016
happy16_df['YEAR'] = 2016
happy16_df['UNIQUE_ID'] = happy16_df['COUNTRY'] + '_' + happy16_df.YEAR.map(str)

# 2017
happy17_df['YEAR'] = 2017
happy17_df['UNIQUE_ID'] = happy17_df['COUNTRY'] + '_' + happy17_df.YEAR.map(str)

In [8]:
#Merge to capture Region and Continent data
happy15_region_df = pd.merge(happy15_df, region_cont_df, on="COUNTRY")
happy16_region_df = pd.merge(happy16_df, region_cont_df, on="COUNTRY")
happy17_region_df = pd.merge(happy17_df, region_cont_df, on="COUNTRY")

In [9]:
# Set index
happy15_region_df.set_index("UNIQUE_ID", inplace=True)
happy16_region_df.set_index("UNIQUE_ID", inplace=True)
happy17_region_df.set_index("UNIQUE_ID", inplace=True)

In [10]:
# Combine all year DF's into one DF
happyall_region_df = pd.concat([happy15_region_df, happy16_region_df, happy17_region_df])

In [11]:
#Reorder fields for final output
all_cols = ['YEAR','COUNTRY','REGION','CONTINENT','HAPPINESS_RANK','HAPPINESS_SCORE','ECONOMY_GDP_PER_CAPITA',
                      'FAMILY','HEALTH_LIFE_EXPECTANCY','FREEDOM','GENEROSITY','TRUST_GOVERNMENT_CORRUPTION','DYSTOPIA_RESIDUAL']
happyall_region_df2= happyall_region_df[all_cols].copy()

In [12]:
happyall_region_df2.head()

,YEAR,COUNTRY,REGION,CONTINENT,HAPPINESS_RANK,HAPPINESS_SCORE,ECONOMY_GDP_PER_CAPITA,FAMILY,HEALTH_LIFE_EXPECTANCY,FREEDOM,GENEROSITY,TRUST_GOVERNMENT_CORRUPTION,DYSTOPIA_RESIDUAL
UNIQUE_ID,,,,,,,,,,,,,
Switzerland_2015,2015,Switzerland,Europe,Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,2.51738
Iceland_2015,2015,Iceland,Europe,Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,2.70201
Denmark_2015,2015,Denmark,Europe,Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,2.49204
Norway_2015,2015,Norway,Europe,Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,2.46531
Canada_2015,2015,Canada,North America,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,2.45176


In [14]:
# load final df into sqlite database
happyall_region_df2.to_sql('happiness_by_region_yr', conn, if_exists='replace', index=True)

In [18]:
pd.read_sql('select * from happiness_by_region_yr', conn)

,UNIQUE_ID,YEAR,COUNTRY,REGION,CONTINENT,HAPPINESS_RANK,HAPPINESS_SCORE,ECONOMY_GDP_PER_CAPITA,FAMILY,HEALTH_LIFE_EXPECTANCY,FREEDOM,GENEROSITY,TRUST_GOVERNMENT_CORRUPTION,DYSTOPIA_RESIDUAL
0,Switzerland_2015,2015,Switzerland,Europe,Europe,1,7.587,1.396510,1.349510,0.941430,0.665570,0.296780,0.419780,2.517380
1,Iceland_2015,2015,Iceland,Europe,Europe,2,7.561,1.302320,1.402230,0.947840,0.628770,0.436300,0.141450,2.702010
2,Denmark_2015,2015,Denmark,Europe,Europe,3,7.527,1.325480,1.360580,0.874640,0.649380,0.341390,0.483570,2.492040
3,Norway_2015,2015,Norway,Europe,Europe,4,7.522,1.459000,1.330950,0.885210,0.669730,0.346990,0.365030,2.465310
4,Canada_2015,2015,Canada,North America,North America,5,7.427,1.326290,1.322610,0.905630,0.632970,0.458110,0.329570,2.451760
5,Finland_2015,2015,Finland,Europe,Europe,6,7.406,1.290250,1.318260,0.889110,0.641690,0.233510,0.413720,2.619550
6,Netherlands_2015,2015,Netherlands,Europe,Europe,7,7.378,1.329440,1.280170,0.892840,0.615760,0.476100,0.318140,2.465700
7,Sweden_2015,2015,Sweden,Europe,Europe,8,7.364,1.331710,1.289070,0.910870,0.659800,0.362620,0.438440,2.371190
8,New Zealand_2015,2015,New Zealand,Asia & Pacific,Oceania,9,7.286,1.250180,1.319670,0.908370,0.639380,0.475010,0.429220,2.264250
9,Australia_2015,2015,Australia,Asia & Pacific,Oceania,10,7.284,1.333580,1.309230,0.931560,0.651240,0.435620,0.356370,2.266460


In [16]:
# read in region continent lookup table
regions = pd.read_csv("region_continent.csv")

In [17]:
# load final lookup table into sqlite database
regions.to_sql('region_continent', conn, if_exists='replace', index=False)

In [19]:
social_progress = pd.read_csv("SocialProgress.csv")

In [20]:
# load Social Progress table into sqlite database
social_progress.to_sql('social_progress', conn, if_exists='replace', index=False)

In [21]:
engine.table_names()

['happiness_by_region_yr', 'region_continent', 'social_progress']